## 模型构造

In [69]:
import torch
from torch import nn
from torch.nn import functional as F
# nn.Sequential定义了一种特殊的Module
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0800,  0.0107,  0.2277, -0.0098,  0.0622, -0.0946, -0.0314, -0.0953,
         -0.0005,  0.0201],
        [ 0.0923,  0.0066,  0.2459, -0.1779,  0.0524, -0.1241,  0.0382,  0.0536,
         -0.0381,  0.0094]], grad_fn=<AddmmBackward0>)

In [70]:
# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [71]:
# 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层
net = MLP()
net(X)

tensor([[ 0.0112, -0.1128, -0.1600, -0.0560, -0.0306, -0.2521, -0.0404,  0.0473,
         -0.2946,  0.0467],
        [-0.1552, -0.1593, -0.1948, -0.0563, -0.1535, -0.2363,  0.0373,  0.0508,
         -0.1925,  0.0369]], grad_fn=<AddmmBackward0>)

In [72]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0295,  0.1846,  0.0376,  0.1247,  0.1873,  0.0092, -0.0804,  0.0460,
          0.0080,  0.2437],
        [-0.1544,  0.1138,  0.0937,  0.0857,  0.0809,  0.0365,  0.0139,  0.2404,
         -0.0563,  0.2623]], grad_fn=<AddmmBackward0>)

In [73]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1446, grad_fn=<SumBackward0>)

In [74]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.0223, grad_fn=<SumBackward0>)

## 参数管理

In [75]:
%reset
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

Nothing done.


tensor([[-0.2104],
        [-0.1985]], grad_fn=<AddmmBackward0>)

In [76]:
# 参数访问
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3305, -0.3315, -0.2215, -0.1980, -0.0525,  0.1360, -0.2632, -0.0607]])), ('bias', tensor([-0.1123]))])


In [77]:
# 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1123], requires_grad=True)
tensor([-0.1123])


In [78]:
net[2].weight.grad == None

True

In [79]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [80]:
net.state_dict()['2.bias'].data

tensor([-0.1123])

In [81]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4182],
        [-0.4182]], grad_fn=<AddmmBackward0>)

In [82]:
# 我们已经设计了网络，让我们看看它是如何组织的
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [83]:
rgnet[0][1][0].bias.data

tensor([-0.1305,  0.4030,  0.0509,  0.4326,  0.4279, -0.4728, -0.4370,  0.4039])

In [84]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
 
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0057, -0.0004,  0.0124, -0.0153]), tensor(0.))

In [85]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [86]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0110, -0.3846, -0.6565,  0.3015])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [87]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -7.5250, -0.0000],
        [-0.0000, -0.0000,  5.1467,  0.0000]], grad_fn=<SliceBackward0>)

In [88]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -6.5250,  1.0000])

In [89]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

In [90]:
%reset
import torch
import torch.nn.functional as F
from torch import nn
# 构造一个没有任何参数的自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

Nothing done.


tensor([-2., -1.,  0.,  1.,  2.])

In [91]:
# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

In [92]:
# 带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-7.4207e-01,  1.8039e+00,  7.1853e-01],
        [ 9.2827e-01,  2.3372e-01, -1.0329e+00],
        [-2.7148e+00, -2.0543e-01,  6.5781e-01],
        [ 1.9205e-01,  2.4328e-01, -3.9972e-02],
        [ 8.4892e-02,  9.0937e-04, -7.6710e-02]], requires_grad=True)

In [93]:
# 使用自定义层直接执行正向传播计算
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.4199],
        [0.0000, 1.2088, 0.0000]])

In [94]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[3.6753],
        [4.1034]])

## 读写文件

In [95]:
%reset
# 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

Nothing done.


tensor([0, 1, 2, 3])

In [96]:
# 存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [97]:
# 写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [98]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [99]:
# 将模型的参数存储为一个叫做“mlp.params”的文件
torch.save(net.state_dict(), 'mlp.params')

In [100]:
# 实例化了原始多层感知机模型的一个备份。 直接读取文件中存储的参数
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [101]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])